In [1]:
import os
import cv2
import argparse
import numpy as np
from skimage import io
from google.colab.patches import cv2_imshow
from PIL import Image


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


## Criando os Bboxes

In [3]:
lista_rotulos_ignorados = [
    'NOME', 'CPF', 'PERMISSÃO', 'ACC', 'CAT. HAB.',
    '1ª HABILITAÇÃO', 'VALIDADE', 'Nº REGISTRO', 'VÁLIDA EM TODO', 'DATA NASCIMENTO',
    'DOC. IDENTIDADE/ÓRG EMISSOR/UF', 'DOC. IDENTIDADE/ÓRG EMISSOR UF', 'DOC IDENTIDADE/ÓRG EMISSOR/UF', 'FILIAÇÃO'
]

def checkClass(name):
    if name == '1ª HABILITAÇÃO':
        return 0
    elif name == 'CAT. HAB.':
        return 1
    elif name == 'CPF':
        return 2
    elif name == 'DATA NASCIMENTO':
        return 3
    elif name == 'DOC. IDENTIDADE/ÓRG EMISSOR/UF' or name == 'DOC. IDENTIDADE/ÓRG EMISSOR UF' or name == 'DOC IDENTIDADE/ÓRG EMISSOR/UF':
        return 4
    elif name == 'FILIAÇÃO':
        return 5
    elif name == 'Nº REGISTRO':
        return 6
    elif name == 'NOME':
        return 7
    elif name == 'VALIDADE':
        return 8
    elif name == 'VÁLIDA EM TODO':
        return 9
    elif name == 'PERMISSÃO':
        return 10
    elif name == 'ACC':
        return 11
    else:
        return -1

def ler_coordenadas_gt(filename):
    coordenadas = []

    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            parts = line.strip().split(',', 4)
            if len(parts) == 5:
                try:
                    x, y, width, height = map(int, parts[:4])
                    rotulo = parts[4].strip()
                    coordenadas.append((x, y, width, height, rotulo))
                except ValueError:
                    print(f"Erro ao processar a linha: {line.strip()}")

    return coordenadas

def cortar_imagens(image_path, coordenadas, codigo, output_dir):
    print(codigo)
    image = Image.open(image_path)
    code_output_dir = os.path.join(output_dir, codigo)
    os.makedirs(code_output_dir, exist_ok=True)

    ultimo_rotulo_ignorado = None
    rotulo_contadores = {}
    quant_permissao = 0

    for coordenada in coordenadas:
        x, y, w, h, rotulo = coordenada

        if rotulo in lista_rotulos_ignorados:
            if rotulo != 'PERMISSÃO':
              ultimo_rotulo_ignorado = rotulo
              continue
            else:
              quant_permissao = quant_permissao + 1
              if quant_permissao == 1:
                ultimo_rotulo_ignorado = rotulo
                continue


        label_id = checkClass(ultimo_rotulo_ignorado)
        if label_id == -1:
            continue

        if label_id not in rotulo_contadores:
            rotulo_contadores[label_id] = 1
        else:
            rotulo_contadores[label_id] += 1

        # Definir o nome do arquivo com base no contador
        contador_atual = rotulo_contadores[label_id]
        file_name = f'{label_id}.{contador_atual}.jpg'

        left = max(x - 5, 0)
        top = max(y - 2, 0)
        right = x + w + 2
        bottom = y + h + 2

        cropped_image = image.crop((left, top, right, bottom))
        if label_id == 9:
            cropped_image = cropped_image.rotate(270, expand=True)
        print(f"Salvando imagem: {file_name}")
        cropped_image.save(os.path.join(code_output_dir, f'{file_name}'))

folder_path = '/content/gdrive/MyDrive/BD/BID Dataset/CNH_Frente'

new_gt = '/content/gdrive/MyDrive/novos_gt'

output_dir = '/content/gdrive/MyDrive/imagens_cortadas'

image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg') and '_gt_segmentation' not in f]

for image_file in image_files:
    image_path = os.path.join(folder_path, image_file)
    codigo_imagem = os.path.splitext(image_file)[0].replace('_in', '')
    gt_file = os.path.join(new_gt, f"{codigo_imagem}_gt_ocr.txt")
    coordenadas = ler_coordenadas_gt(gt_file)
    cortar_imagens(image_path, coordenadas, codigo_imagem, output_dir)

A saída de streaming foi truncada nas últimas 5000 linhas.
Salvando imagem: 0.1.jpg
Salvando imagem: 8.1.jpg
Salvando imagem: 6.1.jpg
Salvando imagem: 9.1.jpg
Salvando imagem: 3.1.jpg
Salvando imagem: 4.1.jpg
Salvando imagem: 4.2.jpg
Salvando imagem: 4.3.jpg
Salvando imagem: 5.1.jpg
Salvando imagem: 5.2.jpg
Salvando imagem: 5.3.jpg
00004122
Salvando imagem: 7.1.jpg
Salvando imagem: 2.1.jpg
Salvando imagem: 1.1.jpg
Salvando imagem: 0.1.jpg
Salvando imagem: 8.1.jpg
Salvando imagem: 6.1.jpg
Salvando imagem: 9.1.jpg
Salvando imagem: 3.1.jpg
Salvando imagem: 4.1.jpg
Salvando imagem: 5.1.jpg
Salvando imagem: 5.2.jpg
Salvando imagem: 5.3.jpg
Salvando imagem: 5.4.jpg
00004137
Salvando imagem: 7.1.jpg
Salvando imagem: 2.1.jpg
Salvando imagem: 1.1.jpg
Salvando imagem: 0.1.jpg
Salvando imagem: 8.1.jpg
Salvando imagem: 6.1.jpg
Salvando imagem: 9.1.jpg
Salvando imagem: 3.1.jpg
Salvando imagem: 4.1.jpg
Salvando imagem: 4.2.jpg
Salvando imagem: 4.3.jpg
Salvando imagem: 5.1.jpg
Salvando imagem: 5.2.jp

In [ ]:
import os

def listar_pastas(caminho_pasta):
    try:
        # Listar todos os arquivos e pastas no caminho especificado
        arquivos_e_pastas = os.listdir(caminho_pasta)

        # Filtrar apenas as pastas
        pastas = [pasta for pasta in arquivos_e_pastas if os.path.isdir(os.path.join(caminho_pasta, pasta))]

        return pastas
    except FileNotFoundError:
        print(f"A pasta '{caminho_pasta}' não foi encontrada.")
        return []
    except PermissionError:
        print(f"Sem permissão para acessar a pasta '{caminho_pasta}'.")
        return []

# Caminho para a pasta imagens_cortadas
pasta_imagens_cortadas = '/content/gdrive/MyDrive/resultados'

# Listar todas as pastas na pasta imagens_cortadas
pastas = listar_pastas(pasta_imagens_cortadas)
print("Pastas encontradas:")
for pasta in pastas:
    print(pasta)


Pastas encontradas:
00003609
00003607
00003608
00003605
00003611
00003614
00003618
00003604
00003615
00003613
00003616
00003617
00003606
00003612
00003610
00003635
00003632
00003627
00003622
00003623
00003629
00003631
00003620
00003630
00003628
00003621
00003634
00003619
00003626
00003633
00003624
00003625
00003636
00003643
00003645
00003640
00003651
00003637
00003644
00003646
00003641
00003642
00003650
00003647
00003648
00003649
00003653
00003639
00003654
00003638
00003652
00003664
00003657
00003666
00003671
00003656
00003669
00003673
00003665
00003672
00003658
00003667
00003660
00003655
00003662
00003661
00003663
00003670
00003659
00003674
00003668
00003685
00003690
00003677
00003675
00003679
00003683
00003687
00003689
00003681
00003676
00003682
00003684
00003688
00003680
00003686
00003678
00003701
00003695
00003693
00003698
00003697
00003699
00003692
00003702
00003696
00003700
00003703
00003691
00003694
